In [1]:
import os

# 클래스별 디렉토리
base_dir = 'project'
class_dirs = [
    'pakimchi.v1i.yolov5pytorch',
    'chonggakkimchi.v1i.yolov5pytorch',
    'Kkakdugi.v1i.yolov5pytorch',
    'Nabak Kimchi.v2i.yolov5pytorch',
    'Gat Kimchi.v1i.yolov5pytorch',
]

print("클래스별 샘플 라벨 확인 결과:\n")

for folder in class_dirs:
    label_dir = os.path.join(base_dir, folder, 'train', 'labels')
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

    if not label_files:
        print(f"❌ {folder}: 라벨 파일 없음")
        continue

    # 첫 번째 라벨 파일 읽기
    sample_path = os.path.join(label_dir, label_files[0])
    with open(sample_path, 'r') as f:
        first_line = f.readline().strip()
        class_id = first_line.split()[0] if first_line else 'empty'

    print(f"✅ {folder} → 샘플 파일: {label_files[0]} → 클래스 ID: {class_id}")


클래스별 샘플 라벨 확인 결과:

✅ pakimchi.v1i.yolov5pytorch → 샘플 파일: Img_039_0299_jpg.rf.9493ab30a69a99adc42a73cc0774db47.txt → 클래스 ID: 0
✅ chonggakkimchi.v1i.yolov5pytorch → 샘플 파일: Img_038_0295_jpg.rf.c2942cce86b1953ed70fbd55eaebe9d8.txt → 클래스 ID: 0
✅ Kkakdugi.v1i.yolov5pytorch → 샘플 파일: Img_030_0222_jpg.rf.fee61a8c209f59eb60559839cb01500b.txt → 클래스 ID: 0
✅ Nabak Kimchi.v2i.yolov5pytorch → 샘플 파일: Img_031_0255_jpg.rf.28aa2941bbfbc3b6da220c84e24b9551.txt → 클래스 ID: 0
✅ Gat Kimchi.v1i.yolov5pytorch → 샘플 파일: Img_029_0255_jpg.rf.f8bd6e34ece9ce88ddc3180605ee9028.txt → 클래스 ID: 0


In [5]:
import os
import shutil

# 클래스별 경로와 ID 매핑
base_dir = 'project'
output_dir = 'yolov5/data/kimchi'
class_dirs = {
    'pakimchi.v1i.yolov5pytorch': 0,
    'chonggakkimchi.v1i.yolov5pytorch': 1,
    'Kkakdugi.v1i.yolov5pytorch': 2,
    'Nabak Kimchi.v2i.yolov5pytorch': 3,
    'Gat Kimchi.v1i.yolov5pytorch': 4,
}

# 출력 디렉토리 생성 (train, val, test)
for split in ['train', 'val', 'test']:
    for sub in ['images', 'labels']:
        os.makedirs(os.path.join(output_dir, split, sub), exist_ok=True)

# 라벨 ID 변경 함수
def update_label_ids(label_path, new_id):
    with open(label_path, 'r') as f:
        lines = f.readlines()
    with open(label_path, 'w') as f:
        for line in lines:
            parts = line.strip().split()
            if parts:  # 빈 라인 방지
                parts[0] = str(new_id)
                f.write(' '.join(parts) + '\n')

# 병합 작업
for folder, class_id in class_dirs.items():
    for split in ['train', 'valid', 'test']:
        img_dir = os.path.join(base_dir, folder, split, 'images')
        lbl_dir = os.path.join(base_dir, folder, split, 'labels')

        # 없는 폴더 건너뜀
        if not os.path.exists(img_dir) or not os.path.exists(lbl_dir):
            print(f"[!] Skipping {folder} - No {split} set found.")
            continue

        out_img_dir = os.path.join(output_dir, split if split != 'valid' else 'val', 'images')
        out_lbl_dir = os.path.join(output_dir, split if split != 'valid' else 'val', 'labels')

        for file in os.listdir(img_dir):
            shutil.copy(os.path.join(img_dir, file), os.path.join(out_img_dir, file))

        for file in os.listdir(lbl_dir):
            label_path = os.path.join(lbl_dir, file)
            update_label_ids(label_path, class_id)
            shutil.copy(label_path, os.path.join(out_lbl_dir, file))

print("\n✅ 병합 완료! yolov5/data/kimchi 디렉토리를 확인하시오오")



✅ 병합 완료! yolov5/data/kimchi 디렉토리를 확인하시오오


In [3]:
import os
from collections import Counter

# 기준 디렉토리
label_root = 'yolov5/data/kimchi'
splits = ['train', 'val', 'test']

# 결과 저장용 딕셔너리
split_class_counts = {}

for split in splits:
    label_dir = os.path.join(label_root, split, 'labels')
    counter = Counter()

    if not os.path.exists(label_dir):
        print(f"[!] {split}/labels 폴더 없음 - 건너뜀.")
        continue

    for fname in os.listdir(label_dir):
        if not fname.endswith('.txt'):
            continue
        with open(os.path.join(label_dir, fname), 'r') as f:
            for line in f:
                parts = line.strip().split()
                if parts:
                    class_id = parts[0]
                    counter[class_id] += 1

    split_class_counts[split] = counter

# 출력
for split in splits:
    print(f"\n📂 {split.upper()} 객체 수:")
    counter = split_class_counts.get(split, {})
    if not counter:
        print("  (없음)")
        continue
    for class_id in sorted(counter.keys(), key=int):
        print(f"  Class {class_id}: {counter[class_id]} objects")



📂 TRAIN 객체 수:
  Class 0: 650 objects
  Class 1: 634 objects
  Class 2: 226 objects
  Class 3: 233 objects
  Class 4: 209 objects

📂 VAL 객체 수:
  Class 0: 139 objects
  Class 1: 136 objects
  Class 2: 48 objects
  Class 3: 49 objects
  Class 4: 45 objects

📂 TEST 객체 수:
  Class 0: 139 objects
  Class 1: 136 objects
  Class 2: 50 objects
  Class 3: 47 objects
  Class 4: 45 objects


In [ ]:
!python -u train.py --img 640 --batch 16 --epochs 50 --data data/kimchi.yaml --weights yolov5s.pt --name kimchi_yolov5s

In [ ]:
!python detect.py --weights runs/train/kimchi_yolov5s/weights/best.pt \
                  --source 검증.mp4 \
                  --conf 0.25 \
                  --project runs/detect --name kimchi_test